# BanDiTS - Breakpoint Detection in Sentinel-1 Time Series

## Import modules
This section is used to import all necessary modules and packages to run the example code

In [ ]:
from BanDiTS import preprocessing, apply_along_axis, export_arr, filter_functions, statistical_functions
from BanDiTS.statistical_functions import *
from BanDiTS.filter_functions import *
from BanDiTS.breakpoint_functions import *
from pathos import multiprocessing as mp
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt


### Define user-dependent input folders and functions to execute
The _main_ function contains all the user input to run the following functions to create the desired outputs

* _raster folder_ is used to specify the input folder location
* _raster_filname_ is used to specify the input folder location

 * This could also be enclosed in a for loop with multiple raster files to work through multiple files automaticly
 
 
* _filter_functions_ is used to specify the desired filter functions to apply
 * this can contain multiple functions in a list
* _filter_args_ contains the arguments for the filter functions (e.g.: [{"kernel": 9}])


* _statistical_functions_ is used to declare the statistical functions to execute, syntax ist equivalent to _filter_functions_
* _statistical_args_ is also equivalent to the _filter_args_ with the difference, that most statistical functions don't require arguments to run. This can be determined by looking at _statistical_functions.py_


* The _break_functions_ module can be used to access further functionionalty for breakpoint detection, it can mostly be applied in a similar fashion as the statistical functions. 


* _output_folder_ is used to specify the output folder location

In [ ]:
def main():
    ###################################     INPUT    ########################################

    # Input Folder:
    raster_folder = ""
    
    # Input File Name
    raster_filename = ""
    

    ###################################     OUTPUT    ########################################

    # Output Folder:
    output_folder = ""

    ####################### USER-DEPENDENT FILTER-FUNCTIONS TO BE USED #######################
    filter_functions = [median_filter]
    filter_args = [{"kernel": 3}]
    # filter_functions = [median_filter, median_filter, median_filter, median_filter, median_filter, median_filter]
    # filter_args = [{"kernel": 3}, {"kernel": 5}, {"kernel": 7}, {"kernel": 13}, {"kernel": 15}, {"kernel": 17}]
    # filter_functions = [sobel_filter, sobel_filter, sobel_filter]
    # filter_args = [{"kernel": [-5, -5, 0, 5, 5]}, {"kernel": [-5, -5, -5, -5, 0, 5, 5, 5, 5]}, {"kernel": [-5, -5, -5, -5, -5, -5, 0, 5, 5, 5, 5, 5, 5]}]

    ##################### USER-DEPENDENT STATISTICAL FUNCTIONS TO BE USED #####################
    statistical_functions = [stdev]
    statistical_args = [{}]

    # Output File Name:
    output_file = raster_filename
    return raster_folder, raster_filename, output_folder, filter_functions, filter_args, statistical_functions, statistical_args
    main_time = datetime.now()
    print("main-time = ", main_time - start_time, "Hr:min:sec")

    ######################   NO USER INPUT BEYOND THIS POINT   ###############################



## Filter function
This function is used for filtering of the time series and returns a filtered 3D stack of time series.
* It applies a for loop to iterate over the _filter_functions_ and produces an individual ouptut for each function

In [ ]:
def filter(raster_folder, raster_filename, output_folder, filter_functions, filter_args):
    start_time = datetime.now()
    input_raster = raster_folder + raster_filename
    hdr_file = "" #input_raster + ".hdr"        # only used for ENVI stacks
    outname = output_folder + raster_filename
    if outname.find(".tif") != -1:
        outname = outname[0:len(outname)-4]


    # arr: full size numpy array 3D XxYxZ 200x300x100
    arr = preprocessing.rio_array(input_raster, hdr_file=hdr_file)
    dates = arr[1]

    for i, func in enumerate(filter_functions):
        kernel_size = str(filter_args[i]['kernel'])
        filtered_arr = apply_along_axis.parallel_apply_along_axis(func1d=func, arr=arr[0], axis=0, cores=mp.cpu_count(),
                                                                  **filter_args[i])
        filtered_arr = np.rollaxis(filtered_arr, 2)
        filtered_arr = np.rollaxis(filtered_arr, 1)
        filtered_arr = np.rollaxis(filtered_arr, 2)


        dtype = type(filtered_arr[0][0][0])
        func_name_end = str(func).find(" at")
        func_name_start = 10
        func_name = str(func)[func_name_start:func_name_end]

        # exporting result to new raster
        export_arr.functions_out_array(outname=outname + "_" + func_name + str(kernel_size), arr=filtered_arr, input_file=input_raster,
                                       dtype=dtype)

    filter_time = datetime.now()
    print("filter-time = ", filter_time - start_time, "Hr:min:sec")

## Statistics function
This function is used extract statistical parameters from the time series and returns the statistical time series values for each pixel in the x- and y-dimension of the stack.
* it applies a for loop to iterate over the _statistical_functions_ and produces an individual ouptut for each function

In [ ]:
def statistics(raster_folder, raster_filename, output_folder, statistical_functions, statistical_args):
    start_time = datetime.now()
    input_raster = raster_folder + raster_filename
    hdr_file = "" #input_raster + ".hdr"        # only used for ENVI stacks
    outname = output_folder + raster_filename
    if outname.find(".tif") != -1:
        outname = outname[0:len(outname)-4]

    # arr: full size numpy array 3D XxYxZ 200x300x100
    arr = preprocessing.rio_array(input_raster, hdr_file=hdr_file)
    dates = arr[1]

    for i, func in enumerate(statistical_functions):
        # creating results with calling wanted algorithm in parallel_apply_along_axis for quick runtime
        result = apply_along_axis.parallel_apply_along_axis(func1d=func, arr=arr[0], axis=0,
                                                            cores=mp.cpu_count(), **statistical_args[i])

        # selecting dtype based on result
        dtype = type(result[0][0])

        func_name_end = str(func).find(" at")
        func_name_start = 10
        func_name = str(func)[func_name_start:func_name_end]

        # exporting result to new raster
        export_arr.functions_out_array(outname=outname + "_" + func_name + str(i), arr=result, input_file=input_raster,
                                       dtype=dtype)

    breakpoint_time = datetime.now()
    print("breakpoint-time = ", breakpoint_time - start_time, "Hr:min:sec")

## Call main function 
This function is used to call the executable functions

In [ ]:
# main func
if __name__ == '__main__':
    start_time = datetime.now()
    in_variables = main()
    filter(raster_folder=str(in_variables[0]), raster_filename=str(in_variables[1]),
           output_folder=str(in_variables[2]), filter_functions=in_variables[3], filter_args=in_variables[4])
    # statistics(raster_folder=str(in_variables[0]), raster_filename=str(in_variables[1]),
    #            output_folder=str(in_variables[2]),  statistical_functions=in_variables[5], statistical_args=in_variables[6])


## Show image
Some example visualization of the generated maps
* _For binary maps use: cmap="binary"_

In [ ]:
%matplotlib notebook
plt.imshow(test[0], cmap="binary")